Copyright (c) Microsoft Corporation. All rights reserved.  
Licensed under the MIT License.

# Using a Trained  FastText Model for Batch Inference

In this notebook, we will demonstrate how to make predictions on large quantities of data asynchronously using the ML pipelines with Azure Machine Learning. Batch inference (or batch scoring) provides cost-effective inference, with unparalleled throughput for asynchronous applications. Batch prediction pipelines can scale to perform inference on terabytes of production data. Batch prediction is optimized for high throughput, fire-and-forget predictions for a large collection of data.

> **Tip**
The dataset we use is not that huge. We aim to make you know the workflow of batch inference. If your system requires low-latency processing (to process a single document or small set of documents quickly), please use realtime inference. Refer to fasttext_realtime_inference.ipynb for more details. 

The outline of this notebook is as follows:

- Create a DataStore referencing documents stored in a blob container.
- Reference a trained fastText model from a complete experiment.
- Use the fastText model to do batch inference on the documents in the data blob container.

## Prerequisites
If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, make sure you go through the configuration Notebook located at https://github.com/Azure/MachineLearningNotebooks first. This sets you up with a working config file that has information on your workspace, subscription id, etc.

In [1]:
import pandas as pd
from azureml.core import Workspace, Dataset, Datastore, Run
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.data.datapath import DataPath
from azureml.core.experiment import Experiment
from azureml.pipeline.wrapper import PipelineRun, Module, dsl
from azureml.pipeline.wrapper._pipeline_endpoint import PipelineEndpoint

### Connect to workspace
Create a workspace object from the existing workspace. Workspace.from_config() reads the file config.json and loads the details into an object named workspace.


In [2]:
workspace = Workspace.from_config('config.json')
print(workspace.name, workspace.resource_group, workspace.location, workspace.subscription_id,
      workspace.compute_targets.keys(), sep='\n')

DesignerDRI_EASTUS
DesignerDRI
eastus
74eccef0-4b8d-4f83-b5f9-fa100d155b22
dict_keys(['attached-aks', 'default', 'compute', 'aml-compute', 'aml-compute-gpu'])


### Create or Attach existing compute resource
By using Azure Machine Learning Compute, a managed service, data scientists can train machine learning models on clusters of Azure virtual machines. Examples include VMs with GPU support. In this tutorial, you create Azure Machine Learning Compute as your training environment. The code below creates the compute clusters for you if they don't already exist in your workspace.

**Creation of compute takes approximately 5 minutes. If the AmlCompute with that name is already in your workspace the code will skip the creation process.**


In [3]:
aml_compute_name = 'aml-compute'
if aml_compute_name in workspace.compute_targets:
    aml_compute = AmlCompute(workspace, aml_compute_name)
    print("Found existing compute target: {}".format(aml_compute_name))
else:
    print("Creating new compute target: {}".format(aml_compute_name))
    provisioning_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2", min_nodes=1, max_nodes=4)
    aml_compute = ComputeTarget.create(workspace, aml_compute_name, provisioning_config)
    aml_compute.wait_for_completion(show_output=True)

Found existing compute target: aml-compute


### Upload the dataset onto a blob container and register it to the workspace

In [4]:
# upload files onto path_on_datastore to a blob container
# our files are in the directory of 'path_on_datastore' in the blob container
path_on_datastore = 'data_for_batch_inference'
datastore = Datastore.get(workspace=workspace, datastore_name='workspaceblobstore')
datastore.upload(src_dir='data/data_for_batch_inference', target_path=path_on_datastore, overwrite=True, show_progress=True)

# get the DataPath object associated with the uploaded dataset
datastore_path = [DataPath(datastore=datastore, path_on_datastore=path_on_datastore)]

# dataset used as the input of the batch inference
dataset = Dataset.File.from_files(path=datastore_path).as_named_input('dataset_for_batch_inference')

Uploading an estimated of 100 files
Uploading data/data_for_batch_inference/0
Uploading data/data_for_batch_inference/1
Uploading data/data_for_batch_inference/10
Uploading data/data_for_batch_inference/11
Uploading data/data_for_batch_inference/12
Uploading data/data_for_batch_inference/13
Uploading data/data_for_batch_inference/14
Uploading data/data_for_batch_inference/15
Uploading data/data_for_batch_inference/16
Uploading data/data_for_batch_inference/17
Uploading data/data_for_batch_inference/18
Uploading data/data_for_batch_inference/19
Uploading data/data_for_batch_inference/2
Uploading data/data_for_batch_inference/20
Uploading data/data_for_batch_inference/21
Uploading data/data_for_batch_inference/22
Uploading data/data_for_batch_inference/23
Uploading data/data_for_batch_inference/24
Uploading data/data_for_batch_inference/25
Uploading data/data_for_batch_inference/26
Uploading data/data_for_batch_inference/27
Uploading data/data_for_batch_inference/28
Uploading data/data_f

Uploaded data/data_for_batch_inference/58, 55 files out of an estimated total of 100
Uploading data/data_for_batch_inference/87
Uploaded data/data_for_batch_inference/59, 56 files out of an estimated total of 100
Uploading data/data_for_batch_inference/88
Uploaded data/data_for_batch_inference/6, 57 files out of an estimated total of 100
Uploading data/data_for_batch_inference/89
Uploaded data/data_for_batch_inference/60, 58 files out of an estimated total of 100
Uploading data/data_for_batch_inference/9
Uploaded data/data_for_batch_inference/61, 59 files out of an estimated total of 100
Uploading data/data_for_batch_inference/90
Uploaded data/data_for_batch_inference/62, 60 files out of an estimated total of 100
Uploading data/data_for_batch_inference/91
Uploaded data/data_for_batch_inference/63, 61 files out of an estimated total of 100
Uploading data/data_for_batch_inference/92
Uploaded data/data_for_batch_inference/64, 62 files out of an estimated total of 100
Uploading data/data_f

### Register an anonymous module from yaml file to the workspace
If you decorate your module function with ```@dsl.module```, azure-cli-ml could help to generate the ```*.spec.yaml``` file.
Please refer to the customized modules for more details.

In [5]:
fasttext_score_module_func = Module.from_yaml(workspace, 'fasttext_score/fasttext_score.spec.yaml')

### Load a trained fastText model from a complete experiment
- get all experiments
- choose an experiment from all experiments
- get the latest run
- get a PipelineRun associated with the run

In [6]:
exp_name_list = [exp.name for exp in Experiment.list(workspace)]
exp_name_list

['sample10',
 'sample5',
 'sample5-realtime',
 'simple10-batch',
 'pythonscript',
 'Data_dependency',
 'clement',
 'new_module',
 'test_module2',
 'test_m',
 'module_SDK_local_module_test',
 'fasttext_pipeline',
 'fasttext_batch_inference',
 'fasttext_pipeline2',
 'fasttext_pipeline_endpoint',
 'split_data_txt']

### Choose the experiment you want with its name

In [7]:
experiment_name = "fasttext_pipeline"
experiment = Experiment(workspace, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
fasttext_pipeline,DesignerDRI_EASTUS,Link to Azure Machine Learning studio,Link to Documentation


In [8]:
# azureml.pipeline.core.run.PipelineRun
run = Run.list(experiment, status='Completed').__next__()
run

Experiment,Id,Type,Status,Details Page,Docs Page
fasttext_pipeline,93878b6e-4648-45d2-9123-d648d14e3872,azureml.PipelineRun,Completed,Link to Azure Machine Learning studio,Link to Documentation


### Get a PipelineRun object

In [9]:
run_id = run.id
# azureml.pipeline.wrapper._pipeline_run.PipelineRun
pipeline_run = PipelineRun(experiment, run_id)
pipeline_run

Experiment,Id,Type,Status,Details Page,Docs Page
fasttext_pipeline,93878b6e-4648-45d2-9123-d648d14e3872,azureml.PipelineRun,Completed,Link to Azure Machine Learning studio,Link to Documentation


### Visualize the pipeline so as to obtain information about the module

In [10]:
pipeline_run.visualize()

<IPython.core.display.Javascript object>

ValidateView(container_id='container_id_7268a475-2dd4-4cfc-9785-10452a05a9f3_widget', env_json='{}', graph_jso…

### Get a StepRun object

In [11]:
# obtain step_run_id from the visualization result.
step_run_id = 'a1b098d2-3352-4974-8155-abb1835258d3'
step_run = pipeline_run.get_step_run(step_run_id)
step_run

Experiment,Id,Type,Status,Details Page,Docs Page
fasttext_pipeline,a1b098d2-3352-4974-8155-abb1835258d3,azureml.StepRun,Completed,Link to Azure Machine Learning studio,Link to Documentation


### In order to use the trained model from a port without registration, we need to install an extra dependence

In [12]:
# Install dataset runtime to enable dataset registration in sample notebooks
!pip install azureml-dataset-runtime[fuse] --extra-index-url https://azuremlsdktestpypi.azureedge.net/modulesdkpreview --user --upgrade

Looking in indexes: https://pypi.org/simple, https://azuremlsdktestpypi.azureedge.net/modulesdkpreview
Requirement already up-to-date: azureml-dataset-runtime[fuse] in /home/azureuser/.local/lib/python3.6/site-packages (1.12.0)


### Use the trained model as the input of a new pipeline

In [12]:
port = step_run.get_port(name='The_better_model')
data_path = port.get_data_path()
model = Dataset.File.from_files(path=[data_path]).as_named_input('model_for_batch_inference')
model

### Construct the pipeline

In [13]:
@dsl.pipeline(name='batch inference', description='Batch Inference', default_compute_target=aml_compute.name)
def training_pipeline(dataset=dataset, model=model):
    fasttext_score = fasttext_score_module_func(
        texts_to_score=dataset,
        fasttext_model_dir=model
    )
    fasttext_score.runsettings.configure(node_count=2, process_count_per_node=2, mini_batch_size="64")

In [14]:
# pipeline
pipeline = training_pipeline()
# pipeline.save(experiment_name=experiment_name)
pipeline

### Run the pipeline

In [15]:
# pipeline_run
experiment_name = 'fasttext_batch_inference'
pipeline_run = pipeline.submit(experiment_name=experiment_name, regenerate_outputs=False)
pipeline_run.wait_for_completion()

Submitted PipelineRun 890cf5de-8037-40cc-859d-215a7a2921e0
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/fasttext_batch_inference/runs/890cf5de-8037-40cc-859d-215a7a2921e0?wsid=/subscriptions/74eccef0-4b8d-4f83-b5f9-fa100d155b22/resourcegroups/DesignerDRI/workspaces/DesignerDRI_EASTUS
PipelineRunId: 890cf5de-8037-40cc-859d-215a7a2921e0
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/fasttext_batch_inference/runs/890cf5de-8037-40cc-859d-215a7a2921e0?wsid=/subscriptions/74eccef0-4b8d-4f83-b5f9-fa100d155b22/resourcegroups/DesignerDRI/workspaces/DesignerDRI_EASTUS


<IPython.core.display.Javascript object>

ValidateView(container_id='container_id_8f2f4246-9ae0-482a-8844-3a24cef107e9_widget', env_json='{}', graph_jso…

<RunStatus.completed: 'Completed'>

### Download results of batch inference

In [16]:
step_run_id = '710b6bd4-e3bc-4cbc-82e5-f1569d303b87'
step_run = pipeline_run.get_step_run(step_run_id)
step_run

Experiment,Id,Type,Status,Details Page,Docs Page
fasttext_batch_inference,710b6bd4-e3bc-4cbc-82e5-f1569d303b87,azureml.StepRun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [17]:
port = step_run.get_port(name='Scored data output dir')
saved_path = port.download(overwrite=True)
saved_path

Downloaded azureml/710b6bd4-e3bc-4cbc-82e5-f1569d303b87/Scored_data_output_dir/136e879ec35445efaa8c140e75934fee.parquet, 1 files out of an estimated total of 4
Downloaded azureml/710b6bd4-e3bc-4cbc-82e5-f1569d303b87/Scored_data_output_dir/a2f5344ac5f642dbb5620d6821c7cffd.parquet, 2 files out of an estimated total of 4
Downloaded azureml/710b6bd4-e3bc-4cbc-82e5-f1569d303b87/Scored_data_output_dir/88e6e74d1e104869a04ea0b7792025f1.parquet, 3 files out of an estimated total of 4
Downloaded azureml/710b6bd4-e3bc-4cbc-82e5-f1569d303b87/Scored_data_output_dir/c6028b5a23194c7ca8eb35dfbd1496f0.parquet, 4 files out of an estimated total of 4


'/tmp/azureml/710b6bd4-e3bc-4cbc-82e5-f1569d303b87/Scored_data_output_dir'

### Check the results of batch inference

In [18]:
df_list = []
for i, file in enumerate(os.listdir(saved_path)):
    path = os.path.join(saved_path, file)
    df_list.append(pd.read_parquet(path, engine='pyarrow'))
df = pd.concat(df_list) 
print(df.shape)
df.head(n=10)

(200, 2)


,Filename,Class
0,/tmp/tmp9rblqy76/0,entertainment
1,/tmp/tmp9rblqy76/1,education
2,/tmp/tmp9rblqy76/10,finance
3,/tmp/tmp9rblqy76/100,game
4,/tmp/tmp9rblqy76/101,education
5,/tmp/tmp9rblqy76/102,society
6,/tmp/tmp9rblqy76/103,game
7,/tmp/tmp9rblqy76/104,education
8,/tmp/tmp9rblqy76/105,entertainment
9,/tmp/tmp9rblqy76/106,game


## Reuse this pipeline with PipelineEndpoint
Suppose you need to do batch inference for a new dataset. Just reuse this pipeline with PipelineEndpoint.

Suppose you want to choose a new model to do batch inference. Just reuse this pipeline with PipelineEndpoint.

### Create a pipeline endpoint
Publish the above pipeline to a pipeline endpoint.

In [19]:
name = 'fasttext_endpoint'
try:
    pipeline_endpoint = PipelineEndpoint.get(workspace=workspace, name=name)
except:
    # If there exists a pipeline endpoint, publish the above pipeline to a pipeline endpoint
    pipeline_endpoint = PipelineEndpoint.publish(workspace=workspace, name=name, pipeline=pipeline_run)
pipeline_endpoint

Name,Description,Date updated,Updated by,Last run time,Last run status,Status,tags,Portal Link
fasttext_endpoint,,2020-08-18 09:37:10.713346+00:00,Xiaoyu Yang,2020-08-18 03:42:13.750136+00:00,Unknown,Unknown,azureml.Designer: true,Link


### Prepare a new dataset
Here, we use the dataset we used just now. To make a difference, we give it a new module name.

In [22]:
dataset = Dataset.File.from_files(path=datastore_path).as_named_input('dataset_for_batch_inference_new')
dataset

### Choose a new model
Here, we use the model used just now. To make a difference, we give it a new module name.

In [23]:
model = Dataset.File.from_files(path=[data_path]).as_named_input('model_for_batch_inference_new')
model

### Submit a pipeline experiment through pipeline parameters

In [25]:
experiment_name = 'fasttext_pipeline_endpoint'
pipeline_run = pipeline_endpoint.submit(experiment_name=experiment_name, parameters={'dataset':dataset, 'model':model})
pipeline_run.wait_for_completion()

Submitted PipelineEndpointRun ead49c22-3621-4a2f-adde-49a789c3d9ce
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/fasttext_pipeline_endpoint/runs/ead49c22-3621-4a2f-adde-49a789c3d9ce?wsid=/subscriptions/74eccef0-4b8d-4f83-b5f9-fa100d155b22/resourcegroups/DesignerDRI/workspaces/DesignerDRI_EASTUS
PipelineRunId: ead49c22-3621-4a2f-adde-49a789c3d9ce
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/fasttext_pipeline_endpoint/runs/ead49c22-3621-4a2f-adde-49a789c3d9ce?wsid=/subscriptions/74eccef0-4b8d-4f83-b5f9-fa100d155b22/resourcegroups/DesignerDRI/workspaces/DesignerDRI_EASTUS


<IPython.core.display.Javascript object>

ValidateView(container_id='container_id_e86065ce-962e-411b-aded-520dd9a12898_widget', env_json='{}', graph_jso…

<RunStatus.completed: 'Completed'>